In [24]:
! pip install --upgrade pip setuptools wheel
! pip install tensorflow
! pip install tensorflow-gpu
! pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
! pip install cupy-cuda11x

  Using cached tensorflow-gpu-2.12.0.tar.gz (2.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [44 lines of output]
      Traceback (most recent call last):
        File "C:\Users\ADMIN\miniconda3\envs\brain_tumor_detection\Lib\site-packages\packaging\requirements.py", line 36, in __init__
          parsed = _parse_requirement(requirement_string)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "C:\Users\ADMIN\miniconda3\envs\brain_tumor_detection\Lib\site-packages\packaging\_parser.py", line 62, in parse_requirement
          return _parse_requirement(Tokenizer(source, rules=DEFAULT_RULES))
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "C:\Users\ADMIN\miniconda3\envs\brain_tumor_detection\Lib\site-packages\packaging\_parser.py", line 80, in _parse_requirement
          url, specifier, marker = _parse_requirement_details(tokenizer)
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

Looking in indexes: https://download.pytorch.org/whl/cu118


In [25]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
import torch
import torch
print("CUDA is available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)
print("Device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU found")



Num GPUs Available:  0
CUDA is available: True
CUDA version: 11.8
Device name: NVIDIA GeForce RTX 3050 Laptop GPU


In [26]:
import cv2
import numpy as np
import os   # To handle directories

In [27]:
INPUT_SIZE = 100
image_directory = 'datasets/'

no_tumor_images=os.listdir(image_directory+ 'no/')
yes_tumor_images=os.listdir(image_directory+ 'yes/')
dataset=[]
label=[]
print(no_tumor_images)
print(yes_tumor_images)

['no0.jpg', 'no1.jpg', 'no10.jpg', 'no100.jpg', 'no1000.jpg', 'no1001.jpg', 'no1002.jpg', 'no1003.jpg', 'no1004.jpg', 'no1005.jpg', 'no1006.jpg', 'no1007.jpg', 'no1008.jpg', 'no1009.jpg', 'no101.jpg', 'no1010.jpg', 'no1011.jpg', 'no1012.jpg', 'no1013.jpg', 'no1014.jpg', 'no1015.jpg', 'no1016.jpg', 'no1017.jpg', 'no1018.jpg', 'no1019.jpg', 'no102.jpg', 'no1020.jpg', 'no1021.jpg', 'no1022.jpg', 'no1023.jpg', 'no1024.jpg', 'no1025.jpg', 'no1026.jpg', 'no1027.jpg', 'no1028.jpg', 'no1029.jpg', 'no103.jpg', 'no1030.jpg', 'no1031.jpg', 'no1032.jpg', 'no1033.jpg', 'no1034.jpg', 'no1035.jpg', 'no1036.jpg', 'no1037.jpg', 'no1038.jpg', 'no1039.jpg', 'no104.jpg', 'no1040.jpg', 'no1041.jpg', 'no1042.jpg', 'no1043.jpg', 'no1044.jpg', 'no1045.jpg', 'no1046.jpg', 'no1047.jpg', 'no1048.jpg', 'no1049.jpg', 'no105.jpg', 'no1050.jpg', 'no1051.jpg', 'no1052.jpg', 'no1053.jpg', 'no1054.jpg', 'no1055.jpg', 'no1056.jpg', 'no1057.jpg', 'no1058.jpg', 'no1059.jpg', 'no106.jpg', 'no1060.jpg', 'no1061.jpg', 'no106

In [28]:
def crop_image(image):
    if image is None:
        print("Lỗi: Không thể đọc tệp ảnh. Hãy kiểm tra lại đường dẫn.")
    else:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)
        edges = cv2.Canny(blurred, 50, 150)
        contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contour = max(contours, key=cv2.contourArea)

        leftmost = tuple(contour[contour[:, :, 0].argmin()][0])
        rightmost = tuple(contour[contour[:, :, 0].argmax()][0])
        topmost = tuple(contour[contour[:, :, 1].argmin()][0])
        bottommost = tuple(contour[contour[:, :, 1].argmax()][0])

        # Xuất tọa độ 4 điểm
        # print("Leftmost point:", leftmost)
        # print("Rightmost point:", rightmost)
        # print("Topmost point:", topmost)
        # print("Bottommost point:", bottommost)
        x_min = min(leftmost[0], rightmost[0])
        x_max = max(leftmost[0], rightmost[0])
        y_min = min(topmost[1], bottommost[1])
        y_max = max(topmost[1], bottommost[1])

        square_size = max(x_max - x_min, y_max - y_min)

        x_center = (x_min + x_max) // 2
        y_center = (y_min + y_max) // 2

        x_min = x_center - square_size // 2
        x_max = x_min + square_size
        y_min = y_center - square_size // 2
        y_max = y_min + square_size

        x_min = max(0, x_min)
        y_min = max(0, y_min)
        x_max = min(image.shape[1], x_max)
        y_max = min(image.shape[0], y_max)

        cropped_image = image[y_min:y_max, x_min:x_max]

        new_size = square_size
        new_image = np.zeros((new_size, new_size, 3), dtype=np.uint8)

        x_offset = (new_size - cropped_image.shape[1]) // 2
        y_offset = (new_size - cropped_image.shape[0]) // 2

        new_image[y_offset:y_offset + cropped_image.shape[0], x_offset:x_offset + cropped_image.shape[1]] = cropped_image
    return new_image



SAVE CROPPED IMAGES

In [29]:
for i , image_name in enumerate(no_tumor_images) :
    image=cv2.imread(image_directory+'no/'+no_tumor_images[i])
    image=crop_image(image)
    image=cv2.resize(image,(INPUT_SIZE,INPUT_SIZE))
    image_name = no_tumor_images[i]
    output_path = os.path.join('datasets/crop_no', image_name)
    cv2.imwrite(output_path, image)


for i , image_name in enumerate(yes_tumor_images) :
    image=cv2.imread(image_directory+'yes/'+yes_tumor_images[i])
    image=crop_image(image)
    image=cv2.resize(image,(INPUT_SIZE,INPUT_SIZE))
    image_name = no_tumor_images[i]
    output_path = os.path.join('datasets/crop_yes', image_name)
    cv2.imwrite(output_path, image)